# 아파트 실거래가 예측 - https://bit.ly/3v5edhJ

### 부동산 빅데이터와 AI를 이용하여 실거래가를 예측 분석 | 회귀 | RMSE

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
print(train_df.shape)
train_df.head()

(1216553, 13)


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [3]:
# 데이터 줄이기

train_df = train_df.loc[train_df["transaction_id"]%50==0, :]
train_df = train_df.reset_index(drop=True)
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,50,12536,서울특별시,인의동,48-2,효성쥬얼리시티,인의동 48-2 효성쥬얼리시티,106.81,2006,200802,11~20,13,64000
2,100,3926,서울특별시,수송동,85,로얄팰리스스위트,수송동 85 로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
3,150,7094,서울특별시,숭인동,569-3,숭인동 105동 (569-3),숭인동 569-3 숭인동 105동 (569-3),59.74,2000,200803,21~31,2,18000
4,200,10147,서울특별시,창신동,703,창신쌍용2,창신동 703 창신쌍용2,106.62,1993,200804,11~20,5,43500


In [4]:
print(test_df.shape)
test_df.head()

(5463, 12)


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,부산광역시,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24320 entries, 0 to 24319
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          24320 non-null  int64  
 1   apartment_id            24320 non-null  int64  
 2   city                    24320 non-null  object 
 3   dong                    24320 non-null  object 
 4   jibun                   24320 non-null  object 
 5   apt                     24320 non-null  object 
 6   addr_kr                 24320 non-null  object 
 7   exclusive_use_area      24320 non-null  float64
 8   year_of_completion      24320 non-null  int64  
 9   transaction_year_month  24320 non-null  int64  
 10  transaction_date        24320 non-null  object 
 11  floor                   24320 non-null  int64  
 12  transaction_real_price  24320 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 2.4+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          5463 non-null   int64  
 1   apartment_id            5463 non-null   int64  
 2   city                    5463 non-null   object 
 3   dong                    5463 non-null   object 
 4   jibun                   5463 non-null   object 
 5   apt                     5463 non-null   object 
 6   addr_kr                 5463 non-null   object 
 7   exclusive_use_area      5463 non-null   float64
 8   year_of_completion      5463 non-null   int64  
 9   transaction_year_month  5463 non-null   int64  
 10  transaction_date        5463 non-null   object 
 11  floor                   5463 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 512.3+ KB


## city

In [7]:
train_df["city"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

In [8]:
test_df["city"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

### Labeling

In [9]:
train_df.loc[train_df["city"] == "서울특별시", "city"] = 0
train_df.loc[train_df["city"] == "부산광역시", "city"] = 1
train_df["city"] = train_df["city"].astype('int')

test_df.loc[test_df["city"] == "서울특별시", "city"] = 0
test_df.loc[test_df["city"] == "부산광역시", "city"] = 1
test_df["city"] = test_df["city"].astype('int')

In [10]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,50,12536,0,인의동,48-2,효성쥬얼리시티,인의동 48-2 효성쥬얼리시티,106.81,2006,200802,11~20,13,64000
2,100,3926,0,수송동,85,로얄팰리스스위트,수송동 85 로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
3,150,7094,0,숭인동,569-3,숭인동 105동 (569-3),숭인동 569-3 숭인동 105동 (569-3),59.74,2000,200803,21~31,2,18000
4,200,10147,0,창신동,703,창신쌍용2,창신동 703 창신쌍용2,106.62,1993,200804,11~20,5,43500


In [11]:
test_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,0,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,1,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,1,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,0,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,1,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


## drop addr_kr

In [12]:
train_df.drop('addr_kr', axis=1, inplace=True)
test_df.drop('addr_kr', axis=1, inplace=True)

## 한강이 보이는 동네

In [13]:
hanriver_dong = ['성수동1가','삼성동','이촌동','공덕동','서교동','한강로3가','목동','고덕동', '암사동', '천호동', '풍납동',
                '잠실동', '청담동', '압구정동', '신사동', '잠원동', '반포동', '흑석동', '노량진동', '여의동', '당산동',
                '양평동', '염창동', '가양동', '방화동']

# isin을 사용해서 한강이 보이는 동네는 1, 아니면 0
# True, False를 integer로 변환

train_df['hanriver']=train_df['dong'].isin(hanriver_dong).astype(int)
test_df['hanriver']=test_df['dong'].isin(hanriver_dong).astype(int)

## 평균 실거래가로 동네 sorting

In [14]:
# "dong"으로 groupby 후, 평균값으로 집계
# 실거래가를 기준으로 오름차순 정렬 후 reset_index()
dong = train_df[['transaction_real_price','dong']].groupby('dong').mean().sort_values('transaction_real_price').reset_index()

# dong의 인덱스를 idx 컬럼에 추가
dong["idx"] = dong.index
dong

,dong,transaction_real_price,idx
0,봉래동5가,4616.666667,0
1,신선동2가,5300.000000,1
2,일광면 삼성리,7740.000000,2
3,보수동2가,7745.000000,3
4,영주동,9707.142857,4
...,...,...,...
403,한남동,125081.578947,403
404,반포동,131895.318471,404
405,회현동2가,148920.000000,405
406,용산동5가,157250.000000,406


In [15]:
# dong_rank에 동 이름을 key, 순서를 value
dong_rank = {}
for i in range(len(dong)):
    dong_rank[dong["dong"].iloc[i]] = dong["idx"].iloc[i]
dong_rank

{'봉래동5가': 0,
 '신선동2가': 1,
 '일광면 삼성리': 2,
 '보수동2가': 3,
 '영주동': 4,
 '대청동1가': 5,
 '동광동5가': 6,
 '토성동2가': 7,
 '기장읍 서부리': 8,
 '금사동': 9,
 '반송동': 10,
 '수정동': 11,
 '감만동': 12,
 '동삼동': 13,
 '영등포동3가': 14,
 '봉래동4가': 15,
 '대청동4가': 16,
 '미근동': 17,
 '구평동': 18,
 '영등포동2가': 19,
 '아미동2가': 20,
 '덕천동': 21,
 '일광면 이천리': 22,
 '학장동': 23,
 '동대신동3가': 24,
 '장림동': 25,
 '기장읍 청강리': 26,
 '서동': 27,
 '명장동': 28,
 '부용동1가': 29,
 '모라동': 30,
 '기장읍 대라리': 31,
 '덕포동': 32,
 '대창동1가': 33,
 '기장읍 동부리': 34,
 '영선동3가': 35,
 '기장읍 대변리': 36,
 '삼선동5가': 37,
 '보수동1가': 38,
 '금곡동': 39,
 '전포동': 40,
 '감천동': 41,
 '남산동': 42,
 '초량동': 43,
 '남항동3가': 44,
 '신평동': 45,
 '당감동': 46,
 '다대동': 47,
 '동대신동1가': 48,
 '우암동': 49,
 '삼락동': 50,
 '토성동5가': 51,
 '괴정동': 52,
 '구포동': 53,
 '만덕동': 54,
 '개금동': 55,
 '장안읍 좌천리': 56,
 '좌천동': 57,
 '주례동': 58,
 '서대신동1가': 59,
 '기장읍 교리': 60,
 '봉래동1가': 61,
 '정관읍 달산리': 62,
 '서대신동2가': 63,
 '흥인동': 64,
 '당리동': 65,
 '회동동': 66,
 '엄궁동': 67,
 '지사동': 68,
 '범천동': 69,
 '수안동': 70,
 '청학동': 71,
 '가야동': 72,
 '하단동': 73,
 '남부민동': 74,
 '광안동': 75,
 '정관읍 방

In [16]:
# df의 dong_rank에 위 dong_rank 딕셔너리 매핑
train_df["dong_rank"] = train_df["dong"].map(dong_rank)
test_df["dong_rank"] = test_df["dong"].map(dong_rank)

# dong 컬럼 drop
train_df.drop("dong", axis=1, inplace=True)
test_df.drop("dong", axis=1, inplace=True)

train_df.head()

,transaction_id,apartment_id,city,jibun,apt,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank
0,0,7622,0,6-13,신현(101동),84.82,2002,200801,21~31,2,37500,0,233
1,50,12536,0,48-2,효성쥬얼리시티,106.81,2006,200802,11~20,13,64000,0,321
2,100,3926,0,85,로얄팰리스스위트,40.07,2005,200803,11~20,10,19000,0,132
3,150,7094,0,569-3,숭인동 105동 (569-3),59.74,2000,200803,21~31,2,18000,0,174
4,200,10147,0,703,창신쌍용2,106.62,1993,200804,11~20,5,43500,0,206


## 평균 실거래가로 아파트 sorting

In [17]:
apt = train_df[['transaction_real_price','apt']].groupby('apt').mean().sort_values('transaction_real_price').reset_index()
apt["idx"] = apt.index


apt_rank = {}
for i in range(len(apt)):
    apt_rank[apt["apt"].iloc[i]] = apt["idx"].iloc[i]

train_df["apt_rank"] = train_df["apt"].map(apt_rank)
test_df["apt_rank"] = test_df["apt"].map(apt_rank)

train_df.drop("apt", axis=1, inplace=True)
test_df.drop("apt", axis=1, inplace=True)

train_df.head()

,transaction_id,apartment_id,city,jibun,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank
0,0,7622,0,6-13,84.82,2002,200801,21~31,2,37500,0,233,3156
1,50,12536,0,48-2,106.81,2006,200802,11~20,13,64000,0,321,3935
2,100,3926,0,85,40.07,2005,200803,11~20,10,19000,0,132,2066
3,150,7094,0,569-3,59.74,2000,200803,21~31,2,18000,0,174,1227
4,200,10147,0,703,106.62,1993,200804,11~20,5,43500,0,206,2731


## 평균 실거래가로 지번 sorting

In [18]:
jibun = train_df[['transaction_real_price','jibun']].groupby('jibun').mean().sort_values('transaction_real_price').reset_index()
jibun["idx"] = jibun.index


jibun_rank = {}
for i in range(len(jibun)):
    jibun_rank[jibun["jibun"].iloc[i]] = jibun["idx"].iloc[i]

train_df["jibun_rank"] = train_df["jibun"].map(jibun_rank)
test_df["jibun_rank"] = test_df["jibun"].map(jibun_rank)

train_df.drop("jibun", axis=1, inplace=True)
test_df.drop("jibun", axis=1, inplace=True)

train_df.head()

,transaction_id,apartment_id,city,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank,jibun_rank
0,0,7622,0,84.82,2002,200801,21~31,2,37500,0,233,3156,2335
1,50,12536,0,106.81,2006,200802,11~20,13,64000,0,321,3935,3315
2,100,3926,0,40.07,2005,200803,11~20,10,19000,0,132,2066,1182
3,150,7094,0,59.74,2000,200803,21~31,2,18000,0,174,1227,1111
4,200,10147,0,106.62,1993,200804,11~20,5,43500,0,206,2731,2469


In [19]:
train_df.head()

,transaction_id,apartment_id,city,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank,jibun_rank
0,0,7622,0,84.82,2002,200801,21~31,2,37500,0,233,3156,2335
1,50,12536,0,106.81,2006,200802,11~20,13,64000,0,321,3935,3315
2,100,3926,0,40.07,2005,200803,11~20,10,19000,0,132,2066,1182
3,150,7094,0,59.74,2000,200803,21~31,2,18000,0,174,1227,1111
4,200,10147,0,106.62,1993,200804,11~20,5,43500,0,206,2731,2469


## 거래 날짜 Labeling

In [20]:
train_df['transaction_date'].unique(), test_df['transaction_date'].unique()

(array(['21~31', '11~20', '1~10', '21~30', '21~28', '21~29'], dtype=object),
 array(['11~20', '21~31', '1~10', '21~30', '21~28'], dtype=object))

In [21]:
date_range = {'1~10': 0, '11~20': 1, '21~28': 2, '21~29': 2, '21~30': 2, '21~31': 2}

train_df.replace({"transaction_date": date_range}, inplace=True)
test_df.replace({"transaction_date": date_range}, inplace=True)

train_df.head()

,transaction_id,apartment_id,city,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank,jibun_rank
0,0,7622,0,84.82,2002,200801,3,2,37500,0,233,3156,2335
1,50,12536,0,106.81,2006,200802,1,13,64000,0,321,3935,3315
2,100,3926,0,40.07,2005,200803,1,10,19000,0,132,2066,1182
3,150,7094,0,59.74,2000,200803,3,2,18000,0,174,1227,1111
4,200,10147,0,106.62,1993,200804,1,5,43500,0,206,2731,2469


In [22]:
train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24320 entries, 0 to 24319
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          24320 non-null  int64  
 1   apartment_id            24320 non-null  int64  
 2   city                    24320 non-null  int64  
 3   exclusive_use_area      24320 non-null  float64
 4   year_of_completion      24320 non-null  int64  
 5   transaction_year_month  24320 non-null  int64  
 6   transaction_date        24320 non-null  int64  
 7   floor                   24320 non-null  int64  
 8   transaction_real_price  24320 non-null  int64  
 9   hanriver                24320 non-null  int64  
 10  dong_rank               24320 non-null  int64  
 11  apt_rank                24320 non-null  int64  
 12  jibun_rank              24320 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 2.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIn

(None, None)

## test_df null 값 처리

In [23]:
test_df["dong_rank"].fillna(test_df["dong_rank"].mean(), inplace=True)
test_df["apt_rank"].fillna(test_df["apt_rank"].mean(), inplace=True)
test_df["jibun_rank"].fillna(test_df["jibun_rank"].mean(), inplace=True)

## 데이터셋 분리

In [24]:
from sklearn.model_selection import train_test_split

x = train_df.drop("transaction_real_price", axis=1)
y = train_df["transaction_real_price"]

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=42)

print(x_train.shape, x_valid.shape)
print(y_train.shape, y_valid.shape)

(17024, 12) (7296, 12)
(17024,) (7296,)


## 모델 학습 및 예측

### SVM

In [25]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)

y_hat = svc.predict(x_valid)

In [26]:
from sklearn.metrics import mean_squared_error

svm_rmse = mean_squared_error(y_valid, y_hat)**0.5
svm_rmse

30975.877455535472

In [27]:
y_test = pd.Series(svc.predict(test_df))

In [30]:
svm_submission = pd.concat([test_df['transaction_id'], y_test], axis=1)

svm_submission.columns = ['transaction_id', 'transaction_real_price']
svm_submission.to_csv('svm_50_submission.csv', header = True, index = False)

In [31]:
test = pd.read_csv('svm_50_submission.csv')
test

,transaction_id,transaction_real_price
0,1145756,20000
1,1198704,20000
2,1222384,20000
3,1179897,20000
4,1223091,20000
...,...,...
5458,1174640,20000
5459,1175575,20000
5460,1157024,20000
5461,1136863,20000


In [32]:
test.transaction_real_price.unique()

array([20000])

### RandomForest

In [33]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=5, random_state=42)
rf.fit(x_train, y_train)


y_hat = rf.predict(x_valid)

In [34]:
rf_rmse = mean_squared_error(y_valid, y_hat)**0.5
rf_rmse

10864.743309020218

In [35]:
y_test = pd.Series(rf.predict(test_df))

In [36]:
rf_submission = pd.concat([test_df['transaction_id'], y_test], axis=1)


rf_submission.columns = ['transaction_id', 'transaction_real_price']
rf_submission.to_csv('rf_50_submission.csv', header = True, index = False)

In [37]:
test = pd.read_csv('rf_50_submission.csv')
test

,transaction_id,transaction_real_price
0,1145756,38736.105774
1,1198704,9958.430493
2,1222384,8100.683599
3,1179897,56360.539055
4,1223091,38552.175777
...,...,...
5458,1174640,48886.623397
5459,1175575,164432.239023
5460,1157024,58855.867291
5461,1136863,38333.015562


In [39]:
len(test["transaction_real_price"].unique())

2145